In [11]:
import os
import clip
import torch

import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, recall_score
from torch.utils.data import DataLoader
from tqdm import tqdm
from torchvision.datasets import ImageFolder

In [12]:
# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)

In [13]:
# Load the dataset
root = os.path.expanduser("~/.cache")

train = ImageFolder('/home/jmunoz/CLIP/PLD_3_Classes_256/Training', transform=preprocess)

test = ImageFolder('/home/jmunoz/CLIP/PLD_3_Classes_256/Testing', transform=preprocess)

validation = ImageFolder('/home/jmunoz/CLIP/PLD_3_Classes_256/Validation', transform=preprocess)


def get_features(dataset):
    all_features = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(DataLoader(dataset, batch_size=100)):
            features = model.encode_image(images.to(device))

            all_features.append(features)
            all_labels.append(labels)

    return torch.cat(all_features).cpu().numpy(), torch.cat(all_labels).cpu().numpy()


# Calculate the image features
train_features, train_labels = get_features(train)
test_features, test_labels = get_features(test)
val_features, val_labels = get_features(validation)

100%|██████████| 5/5 [00:01<00:00,  4.77it/s]


In [33]:
# Perform hyperparameter search
import optuna

# Define an objective function to be minimized.
def objective(trial):

    # Suggest a 'C' value for the trial.
    c = trial.suggest_float('C', 1e-5, 2)

    # Initialize a Logistic Regression model with the suggested 'C' value.
    classifier = LogisticRegression(C=c, random_state=0, max_iter=1000)
    classifier.fit(train_features, train_labels)
    
    predictions = classifier.predict(val_features)

    # Return the mean cross-validated accuracy of the model.
    return f1_score(val_labels, predictions, average='macro')


# Create a study object.
study = optuna.create_study(direction='maximize')

# Optimize the study.
study.optimize(objective, n_trials=100)

# Print the result.
best_params = study.best_params
best_score = study.best_value
print(f"Best score: {best_score}\nBest params: {best_params}")

[I 2024-07-19 08:33:41,781] A new study created in memory with name: no-name-9286a1e0-b4b3-43dc-8f89-00d31b843cb9
[I 2024-07-19 08:33:41,899] Trial 0 finished with value: 0.988227035556161 and parameters: {'C': 1.1158643441588418}. Best is trial 0 with value: 0.988227035556161.
[I 2024-07-19 08:33:42,018] Trial 1 finished with value: 0.988227035556161 and parameters: {'C': 0.6642553767279169}. Best is trial 0 with value: 0.988227035556161.
[I 2024-07-19 08:33:42,149] Trial 2 finished with value: 0.9778537022627285 and parameters: {'C': 0.13778139808952383}. Best is trial 0 with value: 0.988227035556161.
[I 2024-07-19 08:33:42,280] Trial 3 finished with value: 0.9876194027995325 and parameters: {'C': 1.9956351910117194}. Best is trial 0 with value: 0.988227035556161.
[I 2024-07-19 08:33:42,426] Trial 4 finished with value: 0.988227035556161 and parameters: {'C': 0.8715242851213304}. Best is trial 0 with value: 0.988227035556161.
[I 2024-07-19 08:33:42,580] Trial 5 finished with value: 0

Best score: 0.990351663032043
Best params: {'C': 1.8173817875767426}


In [34]:
best_params['C']

1.8173817875767426

In [35]:
# Perform logistic regression
classifier = LogisticRegression(random_state=0, C=best_params['C'], max_iter=1000, verbose=0)
classifier.fit(train_features, train_labels)

LogisticRegression(C=1.8173817875767426, max_iter=1000, random_state=0)

In [36]:
# Evaluate using the logistic regression classifier
predictions = classifier.predict(test_features)
accuracy = np.mean((test_labels == predictions).astype(float)) * 100.
print(f"Accuracy = {accuracy:.3f}")

Accuracy = 98.272


In [37]:
# Calculate F1, precision, recall

f1 = f1_score(test_labels, predictions, average='macro')
precision = precision_score(test_labels, predictions, average='macro')
recall = recall_score(test_labels, predictions, average='macro')

print(f"F1 = {f1:.3f}")
print(f"Precision = {precision:.3f}")
print(f"Recall = {recall:.3f}")

F1 = 0.982
Precision = 0.980
Recall = 0.983
